<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparation</a></span></li><li><span><a href="#Set-up-the-helper-functions" data-toc-modified-id="Set-up-the-helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set up the helper functions</a></span><ul class="toc-item"><li><span><a href="#Tiingo-module---meta-data" data-toc-modified-id="Tiingo-module---meta-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Tiingo module - meta data</a></span></li><li><span><a href="#Price-data" data-toc-modified-id="Price-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Price data</a></span></li><li><span><a href="#Daily-fundamental-data" data-toc-modified-id="Daily-fundamental-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Daily fundamental data</a></span></li><li><span><a href="#Get-quarterly-fundamental" data-toc-modified-id="Get-quarterly-fundamental-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Get quarterly fundamental</a></span></li><li><span><a href="#Filter-Stocks" data-toc-modified-id="Filter-Stocks-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Filter Stocks</a></span></li><li><span><a href="#Fetching-data" data-toc-modified-id="Fetching-data-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Fetching data</a></span></li></ul></li><li><span><a href="#We're-almost-set" data-toc-modified-id="We're-almost-set-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>We're almost set</a></span><ul class="toc-item"><li><span><a href="#Get-the-stock-symbols" data-toc-modified-id="Get-the-stock-symbols-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get the stock symbols</a></span></li><li><span><a href="#Start-downloading-the-data-we-need" data-toc-modified-id="Start-downloading-the-data-we-need-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Start downloading the data we need</a></span></li></ul></li></ul></div>

## Preparation

In this notebook, we're going to walk through the detail of how to prepare the data we needed to implement the factor analysis that we describe in the previous series [Vol. 1. Introduction the idea of factor analysis](https://mikelhsia.github.io/2021/01/31/2021-01-31-factor-analysis/#more).

First of all, let's start with importing all the libraries that we're going to use throughout this series

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime

import statsmodels.api as sm
import scipy
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import notebook

# Here we configure the option to better display the detail in pd.DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)

`dotenv` is a nice library that if you want to conceal some sensitive data in `.env` as environment variables. Here As I'm going to use my own Tiingo account to demonstrate, I hid the API token in the `.env` file and load it with `dotenv`

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), verbose=True, override=True)
TIINGO_API_TOKEN = os.getenv('TIINGO_API_TOKEN')

## Set up the helper functions

We are using [Tiingo](https://api.tiingo.com/) as our data source here. To understand how we transform the raw data from API returns to what we need, let's take a look at what Tiingo provides us:

### Tiingo module - meta data

Set up the header that we're going to use in the requests

In [4]:
__headers = {
    'Content-Type': 'application/json'
}
__token = TIINGO_API_TOKEN

**metadata** is a gigantic table that consists of basic info of all available stocks. There are only a couple of fields that we care about throughout the series:
- **ticker**: the symbol of the stocks
- **isActive**: whether this stock(company) is still tradable
- **isADR**: whether this stock is an ADR
- **sector**: what kind of business this company is running
- **sicCode**: SIC stands for `Standard Industrial Classification`

Since this endpoint takes an extraordinary long time to get a response, we can store the data on our local machine so that we can load the metadata file faster.

We're going to add the `sector` column from metadata to our final DataFrame

In [5]:
def update_stock_meta_data_to_file():
    fundamental_meta_url = 'https://api.tiingo.com/tiingo/fundamentals/meta'
    fundamental_meta_data_path = f'{os.path.dirname(__file__)}/data/meta.json'

    try:
        fundamental_meta_data = requests.get(
            url=fundamental_meta_url,
            headers=__headers,
            timeout=240,
        )
        if fundamental_meta_data.status_code == 200:
            json.loads(fundamental_meta_data.text)
    except Exception as e:
        print(e)
        raise(e)
    else:
        with open(fundamental_meta_data_path, 'w+') as file:
            file.write(fundamental_meta_data.text)
        return

    print('Fundamental meta API failed.')
    return

def load_meta() -> str:
    fundamental_meta_data_path = f'./data/meta.json'
    with open(fundamental_meta_data_path, 'r') as fp:
        content=fp.read()
    content = json.loads(content)
    return content

In [9]:
meta_df = pd.json_normalize(load_meta())
meta_df.head()

,permaTicker,ticker,name,isActive,isADR,sector,industry,sicCode,sicSector,sicIndustry,reportingCurrency,location,companyWebsite,secFilingWebsite,statementLastUpdated,dailyLastUpdated
0,US000000000247,a,Agilent Technologies Inc,True,False,Healthcare,Diagnostics & Research,3826.0,Manufacturing,Laboratory Analytical Instruments,usd,"California, USA",http://www.agilent.com,https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001090872,2020-12-23T23:02:28.702Z,2020-12-30T21:43:23.916Z
1,US000000000091,aa,Alcoa Corp,True,False,Basic Materials,Aluminum,3350.0,Manufacturing,Rolling Drawing & Extruding Of Nonferrous Metals,usd,"New York, USA",http://www.alcoa.com,https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001675149,2020-11-04T23:02:02.578Z,2020-12-30T21:43:24.024Z
2,US000000057010,aaab,ADMIRALTY BANCORP INC,False,False,Financial Services,Banks - Regional,6022.0,Finance Insurance And Real Estate,State Commercial Banks,usd,"Florida, USA",None,https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001066808,2019-12-06T00:59:50.670Z,2019-12-06T00:59:50.670Z
3,US000000067899,aaagy,ALTANA AKTIENGESELLSCHAFT Foreign,False,True,Healthcare,Biotechnology,2834.0,Manufacturing,Pharmaceutical Preparations,eur,Jordan,None,https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001182802,2019-12-06T01:00:13.855Z,2020-05-21T15:08:05.197Z
4,US000000013210,aaap,Advanced Accelerator Applications SA,False,True,Healthcare,Biotechnology,2834.0,Manufacturing,Pharmaceutical Preparations,eur,France,None,https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001611787,2020-10-13T22:01:34.100Z,2020-10-13T22:01:45.062Z


### Price data
Here's the API that requests the pricing data of a specific symbol

In [10]:
def get_price(symbol, start_date=None, end_date=None, resampleFreq='daily'):
    
    start_param = 'startDate={}'.format(start_date) if start_date is not None else ''
    end_param = '&endDate={}'.format(end_date) if end_date is not None else ''
    pricing_daily_url = 'https://api.tiingo.com/tiingo/daily/{}/prices?token={}&{}{}&resampleFreq={}'.format(symbol, __token, start_param, end_param, resampleFreq)

    try:
        pricing_daily_data = requests.get(
            url=pricing_daily_url,
            headers=__headers
        )
    except Exception as e:
        raise(e)
    else:
        if pricing_daily_data.status_code == 200:
            return pricing_daily_data.json()
    print('[{}] "pricing_daily_url" doesn\'t return successfully'.format(symbol))
    return None

In [13]:
pd.DataFrame(get_price('aapl', '2018-07-31', '2021-01-31')).head()

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2020-12-31T00:00:00.000Z,132.69,134.7400,131.720,134.08,99116586,132.491399,134.538330,131.522850,133.879318,99116586,0.0,1.0
1,2021-01-04T00:00:00.000Z,129.41,133.6116,126.760,133.52,143301887,129.216308,133.411619,126.570274,133.320156,143301887,0.0,1.0
2,2021-01-05T00:00:00.000Z,131.01,131.7400,128.430,128.89,97664898,130.813913,131.542820,128.237775,128.697086,97664898,0.0,1.0
3,2021-01-06T00:00:00.000Z,126.60,131.0499,126.382,127.72,155087970,126.410514,130.853753,126.192840,127.528837,155087970,0.0,1.0
4,2021-01-07T00:00:00.000Z,130.92,131.6300,127.860,128.36,109578157,130.724048,131.432985,127.668628,128.167879,109578157,0.0,1.0


### Daily fundamental data
This is the API that we're getting the `marketCap`, `peRatio`, `pbRatio` from.

In [14]:
def get_daily_fundamental(symbol, start_date=None, end_date=None):
    
    start_param = 'startDate={}'.format(start_date) if start_date is not None else ''
    end_param = '&endDate={}'.format(end_date) if end_date is not None else ''
    fundamental_daily_fundamental_url = 'https://api.tiingo.com/tiingo/fundamentals/{}/daily?token={}&{}{}'.format(symbol, __token, start_param, end_param)
    
    try:
        fundamental_daily_fundamental_data = requests.get(
            url=fundamental_daily_fundamental_url,
            headers=__headers
        )
    except Exception as e:
        raise(e)
    else:
        if fundamental_daily_fundamental_data.status_code == 200:
            return fundamental_daily_fundamental_data.json()
    print('[{}] "fundamental_daily_fundamental_url" doesn\'t return successfully'.format(symbol))
    return None

In [16]:
pd.DataFrame(get_daily_fundamental('aapl', '2018-07-31', '2021-01-31')).head()

,date,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2020-12-30T00:00:00.000Z,2.273481e+12,2.349514e+12,35.706275,34.330167,1.022498
1,2020-12-31T00:00:00.000Z,2.255969e+12,2.332002e+12,35.431242,34.065733,1.014622
2,2021-01-04T00:00:00.000Z,2.200203e+12,2.276236e+12,34.555407,33.223653,0.989541
3,2021-01-05T00:00:00.000Z,2.227406e+12,2.303439e+12,34.982644,33.634424,1.001776
4,2021-01-06T00:00:00.000Z,2.152428e+12,2.228461e+12,33.805073,32.502237,0.968054


### Get quarterly fundamental
This is the API that we get data from the official fiscal quarterly report from.

In [18]:
def get_quarterly_fundamental(symbol, start_date=None, end_date=None):
    
    start_param = 'startDate={}'.format(start_date) if start_date is not None else ''
    end_param = '&endDate={}'.format(end_date) if end_date is not None else ''
    fundamental_quarterly_url = 'https://api.tiingo.com/tiingo/fundamentals/{}/statements?token={}&{}{}'.format(symbol, __token, start_param, end_param)
    
    try:
        fundamental_quarterly_data = requests.get(
            url=fundamental_quarterly_url,
            headers=__headers
        )
    except Exception as e:
        raise(e)
    else:
        if fundamental_quarterly_data.status_code == 200:
            return fundamental_quarterly_data.json()
    print('[{}] "fundamental_quarterly_url" doesn\'t return successfully'.format(symbol))
    return None

def normalize_fundamental_quarterly_data(data):
    """[summary]

    Args:
        data ([type]): [description]

    Returns:
        [type]: [description]
    """
    df = pd.json_normalize(data)

    sheets = [
        'statementData.balanceSheet',
        'statementData.cashFlow',
        'statementData.incomeStatement',
        'statementData.overview'
    ]

    for col in sheets:
        if col not in df.columns:
            continue

        fdf = None
        for i in df.index:
            dic = {}

            if df.iloc[i][col] is np.NaN:
                continue
            for item in df.iloc[i][col]:
                code = None
                value = None
                for k in item.keys():
                    if k == 'dataCode':
                        code = item[k]
                    if k == 'value':
                        value = item[k]
                dic[code] = value
            data = [x for x in dic.values()]
            cols = [col + '.' + x for x in dic.keys()]

            if fdf is None:
                fdf = pd.DataFrame(np.array([data]), index=[i], columns=cols)
            else:
                fdf = fdf.append(pd.DataFrame(np.array([data]), index=[i], columns=cols))
        df = pd.concat([df, fdf], axis=1)
        df = df.drop([col], axis=1)

    df['date'] = df['date'].astype('datetime64[ns]')
    return df

In [25]:
pd.DataFrame(get_quarterly_fundamental('aapl', '2020-07-31', '2021-01-31')).head()

,date,statementData,quarter,year
0,2020-12-26,"{'balanceSheet': [{'value': 199948000000.0, 'dataCode': 'assetsNonCurrent'}, {'value': 0.0, 'dat...",1,2021
1,2020-09-26,"{'balanceSheet': [{'value': 323888000000.0, 'dataCode': 'totalAssets'}, {'value': 17102536000.0,...",4,2020
2,2020-09-26,"{'balanceSheet': [{'value': 17102536000.0, 'dataCode': 'sharesBasic'}, {'value': 0.0, 'dataCode'...",0,2020


Unfortunately, the data returned from API is muti-leveled with dictionary embedded, which we want them to be in the same level so that we can process them with one single DataFrame. Therefore, we also need a function to normalize the data into parsable json. See what we did in `normalize_fundamental_quarterly_data()`.

In [23]:
normalize_fundamental_quarterly_data(get_quarterly_fundamental('aapl', '2020-07-31', '2021-01-31')).head()

,date,quarter,year,statementData.balanceSheet.assetsNonCurrent,statementData.balanceSheet.taxAssets,statementData.balanceSheet.intangibles,statementData.balanceSheet.totalLiabilities,statementData.balanceSheet.investments,statementData.balanceSheet.assetsCurrent,statementData.balanceSheet.debtCurrent,statementData.balanceSheet.retainedEarnings,statementData.balanceSheet.accoci,statementData.balanceSheet.liabilitiesNonCurrent,statementData.balanceSheet.cashAndEq,statementData.balanceSheet.acctRec,statementData.balanceSheet.sharesBasic,statementData.balanceSheet.liabilitiesCurrent,statementData.balanceSheet.acctPay,statementData.balanceSheet.debt,statementData.balanceSheet.investmentsCurrent,statementData.balanceSheet.deferredRev,statementData.balanceSheet.totalAssets,statementData.balanceSheet.taxLiabilities,statementData.balanceSheet.investmentsNonCurrent,statementData.balanceSheet.debtNonCurrent,statementData.balanceSheet.equity,statementData.balanceSheet.deposits,statementData.balanceSheet.ppeq,statementData.balanceSheet.inventory,statementData.cashFlow.issrepayEquity,statementData.cashFlow.ncfx,statementData.cashFlow.ncfo,statementData.cashFlow.freeCashFlow,statementData.cashFlow.depamor,statementData.cashFlow.issrepayDebt,statementData.cashFlow.ncf,statementData.cashFlow.sbcomp,statementData.cashFlow.businessAcqDisposals,statementData.cashFlow.capex,statementData.cashFlow.payDiv,statementData.cashFlow.investmentsAcqDisposals,statementData.cashFlow.ncff,statementData.cashFlow.ncfi,statementData.incomeStatement.opex,statementData.incomeStatement.netinc,statementData.incomeStatement.ebitda,statementData.incomeStatement.prefDVDs,statementData.incomeStatement.intexp,statementData.incomeStatement.shareswa,statementData.incomeStatement.grossProfit,statementData.incomeStatement.netIncDiscOps,statementData.incomeStatement.revenue,statementData.incomeStatement.netIncComStock,statementData.incomeStatement.rnd,statementData.incomeStatement.costRev,statementData.incomeStatement.epsDil,statementData.incomeStatement.sga,statementData.incomeStatement.shareswaDil,statementData.incomeStatement.taxExp,statementData.incomeStatement.ebit,statementData.incomeStatement.consolidatedIncome,statementData.incomeStatement.eps,statementData.incomeStatement.ebt,statementData.incomeStatement.opinc,statementData.incomeStatement.nonControllingInterests,statementData.overview.roa,statementData.overview.grossMargin,statementData.overview.bvps,statementData.overview.roe,statementData.overview.rps,statementData.overview.currentRatio,statementData.overview.profitMargin,statementData.overview.piotroskiFScore,statementData.overview.bookVal,statementData.overview.longTermDebtEquity,statementData.overview.revenueQoQ,statementData.overview.epsQoQ
0,2020-12-26,1,2021,1.999480e+11,0.0,0.0,2.878300e+11,1.595610e+11,1.541060e+11,1.276200e+10,1.430100e+10,179000000.0,1.553230e+11,3.601000e+10,5.862000e+10,1.700180e+10,1.325070e+11,6.384600e+10,1.120430e+11,4.081600e+10,7.395000e+09,3.540540e+11,0.0,1.187450e+11,9.928100e+10,6.622400e+10,0.0,3.793300e+10,4.973000e+09,-2.477500e+10,0.0,3.876300e+10,3.526300e+10,2.666000e+09,-9.780000e+08,-2.070000e+09,2.020000e+09,-9.000000e+06,-3.500000e+09,-3.613000e+09,-5.279000e+09,-3.224900e+10,-8.584000e+09,1.079400e+10,2.875500e+10,3.624500e+10,0.0,0.0,1.693512e+10,4.432800e+10,0.0,1.114390e+11,2.875500e+10,5.163000e+09,6.711100e+10,1.68,5.631000e+09,1.711369e+10,4.824000e+09,3.357900e+10,2.875500e+10,1.700,3.357900e+10,3.353400e+10,0.0,0.194362,0.397778,3.895117,0.905941,6.554541,1.163003,0.397778,7.0,6.622400e+10,1.499169,0.213681,0.349206
1,2020-09-26,4,2020,1.801750e+11,0.0,0.0,2.585490e+11,1.538140e+11,1.437130e+11,1.376900e+10,1.496600e+10,-406000000.0,1.531570e+11,3.801600e+10,3.744500e+10,1.710254e+10,1.053920e+11,4.229600e+10,1.124360e+11,5.292700e+10,6.643000e+09,3.238880e+11,0.0,1.008870e+11,9.866700e+10,6.533900e+10,0.0,3.676600e+10,4.061000e+09,-1.673700e+10,0.0,2.057600e+10,1.879200e+10,2.702000e+09,-7.030000e+08,4.75

### Filter Stocks
- Context goes here. [【How 2】 Vol. 4. How to produce the S&P 500 Historical Components & Changes](https://mikelhsia.github.io/2021/02/15/2021-02-15-how2-snp500-historic-composition/)

In [4]:
def cleanup_stock_list(stock_list) -> list:
    stock_list = [s.lower() for s in stock_list]
    stock_list = [s if s != 'brk.b' else 'brk-b' for s in stock_list]
    stock_list = [s if s != 'bf.b' else 'bf-b' for s in stock_list]
    stock_list = [s if s != 'chk' else 'chkaq' for s in stock_list]
    stock_list = [s if s != 'dps' else 'kdp' for s in stock_list]
    
    # WYN spun-off to WH
    stock_list = [s if s != 'wyn' else 'wh' for s in stock_list]

    # Fundamental not found
    stock_list = [s for s in stock_list if s not in ['frc']]

    # Class B or C
    # goog => googl
    # ua => uaa
    # fox => foxa
    # nws => nwsa
    # disck => disca
    stock_list = [s for s in stock_list if s not in ['ua','fox', 'goog', 'nws', 'disck']]

    # Stock was acquired by another company @ 2018-01-03
    stock_list = [s for s in stock_list if s not in ['bcr']]

    # Stock fundamental is only available before 2018-01-01
    stock_list = [s for s in stock_list if s not in ['csra', 'sni']]
    
    # Symbol changed
    stock_list = [s for s in stock_list if s not in ['dwdp']]
    return stock_list

In [ ]:
def get_current_snp_tickers() -> list:
    _header = {
        'authority': 'www.slickcharts.com',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    res = requests.get('https://www.slickcharts.com/sp500', headers=_header)
    snp500_current = pd.read_html(res.content, index_col=0)
    
    # The latest S&P 500 composition (https://www.slickcharts.com/sp500)
    l = snp500_current[0].Symbol.to_list()
    
    return l

In [ ]:
def get_snp_stock_scaffolding(start_date, end_date, freq='daily'):
    data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    # Get current S&P table and set header column
    sp500 = data[0].iloc[1:,[0,1,6,7]]
    columns = ['added_ticker', 'name', 'date', 'cik']
    sp500.columns = columns
    sp500.loc[sp500['date'].isnull(), 'date'] = '1957-01-01'

    # One date is in the wrong format. Correcting it.
    sp500.loc[:,'date'] = sp500.loc[:,'date'].apply(lambda x: datetime.datetime.strptime(x[:10],'%Y-%m-%d'))
    sp500 = pd.melt(sp500, id_vars=['date', 'name', 'cik'], value_vars=['added_ticker'])

    sp500_adjustments = data[1]
    sp500_adjustments = sp500_adjustments[2:].copy()
    columns = ['date', 'added_ticker', 'added_name', 'removed_ticker', 'removed_name', 'reason']
    sp500_adjustments.columns = columns
    updates = sp500_adjustments[~sp500_adjustments['date'].str.contains(',')].T.shift(1).T
    sp500_adjustments['date'].loc[~sp500_adjustments['date'].str.contains(',')] = np.nan
    sp500_adjustments[sp500_adjustments['added_ticker'].isnull()]
    sp500_adjustments.update(updates)
    sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
    sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
    sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
    sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
    sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
    sp500_additions = sp500_adjustments[~sp500_adjustments['added_ticker'].isnull()]
    sp500_additions = sp500_additions[['date', 'added_ticker', 'added_name']]
    sp500_additions.rename(columns={'added_name': 'name'}, inplace=True)
    sp500_additions = pd.melt(sp500_additions, id_vars=['date','name'], value_vars=['added_ticker'])
    sp500_deletions = sp500_adjustments[~sp500_adjustments['removed_ticker'].isnull()]
    sp500_deletions = sp500_deletions[['date', 'removed_ticker', 'removed_name']]
    sp500_deletions.rename(columns={'removed_name': 'name'}, inplace=True)
    sp500_deletions = pd.melt(sp500_deletions, id_vars=['date','name'], value_vars=['removed_ticker'])
    sp500_history = pd.concat([sp500_deletions, sp500_additions])
    
    df = pd.concat([sp500, sp500_history], ignore_index=True)
    df['date'] = pd.to_datetime(df['date'], utc=True)
    df.sort_values(by='cik', ascending=False, inplace=True)
    deduped_df = df[~df.duplicated(['date', 'variable', 'value'])].copy()    
    deduped_df = deduped_df.sort_values(['date', 'variable']).set_index(['date', 'variable'])

    l = get_current_snp_tickers()
    dictionary = {}

    for d in deduped_df.index.levels[0][::-1]:
        if 'added_ticker' in deduped_df.loc[pd.to_datetime(d),:].index:
            removed_stock_list = [s.upper() for s in deduped_df.loc[(pd.to_datetime(d), 'added_ticker'), 'value'].values]
        else:
            removed_stock_list = []

        if 'removed_ticker' in deduped_df.loc[pd.to_datetime(d),:].index:
            added_stock_list = [s.upper() for s in deduped_df.loc[(pd.to_datetime(d), 'removed_ticker'), 'value'].values]
        else:
            added_stock_list = []

        l = [x.upper() for x in (l + added_stock_list) if x not in removed_stock_list]
        l = list(set(l))
        dictionary[d] = l

        if d <= pd.to_datetime(START_DATE, utc=True):
            break

    if freq == 'daily':
        # Business Day
        re = pd.DataFrame(index=pd.to_datetime(pd.date_range(start_date, end_date, freq = 'B'), utc=True))
    elif freq == 'weekly':
        # business month start frequency
        re = pd.DataFrame(index=pd.to_datetime(pd.date_range(START_DATE,END_DATE, freq = 'BMS'), utc=True))

    re['ticker'] = pd.DataFrame([dictionary]).T
    re = re.ffill().dropna()
    re = re.explode('ticker').reset_index().sort_values(['index', 'ticker'])
    re['ticker'] = re['ticker'].str.lower()
    re = re.set_index(['index', 'ticker'])
    re.index.rename(['date', 'ticker'], inplace=True)
    return re

### Fetching data
Here we're going to build a function for downloading data from Tiingo, processing them into one `daily available data` and another `quarterly available data`, and then storing the results into two separate CSV files. Here's the step-to-step illustration:

First of all, download price data, daily fundamental data, and quarterly fundamental data as we needed. 
- Remember to normalize the quarterly fundamental data so that we can melt the data into 2D arrays.
- One thing to be noted, that the row is duplicated when its `quarter` equals 0 in quarterly fundamental data. We are going to remove this row in order to retain the clean data.
- Let's don't forget to specify the type in the `date` column.

In [28]:
df = pd.DataFrame(get_price('aapl', '2018-07-31', '2021-01-31'))
df2 = pd.DataFrame(get_daily_fundamental('aapl', '2018-07-31', '2021-01-31'))
df3 = normalize_fundamental_quarterly_data(get_quarterly_fundamental('aapl', '2018-07-31', '2021-01-31'))
df3 = df3[~df3['quarter'].isin([0])]    # Remove the records that quarter = 0

df['date'] = pd.to_datetime(df['date'], utc=True)
df2['date'] = pd.to_datetime(df2['date'], utc=True)
df3['date'] = pd.to_datetime(df3['date'], utc=True)

Let's start with merging these tables into one big sheet!

One thing we need to be aware of all the time is that the fiscal data would only be available for the next day when the companies published the fiscal reports. So let's `ffill()` for the data we have acquired.

In [32]:
dd = pd.merge(df, df2, on='date', how='left')
dd = dd.ffill()
df3 = df3.ffill()
df3 = df3[~df3['quarter'].isin([0])]

In [35]:
dd.head(10)

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2018-07-31 00:00:00+00:00,190.29,192.14,189.34,190.30,39373038,46.070749,46.518649,45.840746,46.073170,157492152,0.00,1.0,9.353016e+11,1.017931e+12,17.112410,8.136666,0.422777
1,2018-08-01 00:00:00+00:00,201.50,201.76,197.31,199.13,67935716,48.784781,48.847729,47.770348,48.210984,271742864,0.00,1.0,9.732301e+11,1.055859e+12,18.120504,8.615998,0.447683
2,2018-08-02 00:00:00+00:00,207.39,208.38,200.35,200.58,62404012,50.210797,50.450484,48.506356,48.562041,249616048,0.00,1.0,1.001678e+12,1.084307e+12,18.650180,8.867850,0.460769
3,2018-08-03 00:00:00+00:00,207.99,208.74,205.48,207.03,33447396,50.356062,50.537643,49.748371,50.123638,133789584,0.00,1.0,1.004576e+12,1.087205e+12,18.704137,8.893505,0.462102
4,2018-08-06 00:00:00+00:00,209.07,209.25,207.07,208.00,25425387,50.617539,50.661118,50.133323,50.358483,101701548,0.00,1.0,1.009793e+12,1.092422e+12,18.801259,8.939685,0.464502
5,2018-08-07 00:00:00+00:00,207.11,209.50,206.76,209.32,25587387,50.143007,50.721645,50.058269,50.678066,102349548,0.00,1.0,1.000326e+12,1.082955e+12,18.625000,8.855877,0.460147
6,2018-08-08 00:00:00+00:00,207.25,207.81,204.52,206.05,22525487,50.176902,50.312483,49.515947,49.886372,90101948,0.00,1.0,1.001002e+12,1.083631e+12,18.637590,8.861864,0.460458
7,2018-08-09 00:00:00+00:00,208.88,209.78,207.20,207.28,23492626,50.571538,50.789436,50.164797,50.184165,93970504,0.00,1.0,1.008875e+12,1.091504e+12,18.784173,8.931561,0.464080
8,2018-08-10 00:00:00+00:00,207.53,209.10,206.67,207.36,24611202,50.421431,50.802878,50.212486,50.380128,98444808,0.73,1.0,1.002355e+12,1.084984e+12,18.662770,8.873836,0.461080
9,2018-08-13 00:00:00+00:00,208.87,210.95,207.70,207.70,25890880,50.746997,51.252354,50.462734,50.462734,103563520,0.00,1.0,1.008827e+12,1.091456e+12,18.783273,8.931134,0.464057


In [36]:
df3.head(10)

,date,quarter,year,statementData.balanceSheet.accoci,statementData.balanceSheet.investmentsNonCurrent,statementData.balanceSheet.taxAssets,statementData.balanceSheet.retainedEarnings,statementData.balanceSheet.investmentsCurrent,statementData.balanceSheet.debtCurrent,statementData.balanceSheet.totalAssets,statementData.balanceSheet.assetsNonCurrent,statementData.balanceSheet.assetsCurrent,statementData.balanceSheet.cashAndEq,statementData.balanceSheet.deferredRev,statementData.balanceSheet.equity,statementData.balanceSheet.ppeq,statementData.balanceSheet.debtNonCurrent,statementData.balanceSheet.inventory,statementData.balanceSheet.acctPay,statementData.balanceSheet.investments,statementData.balanceSheet.debt,statementData.balanceSheet.taxLiabilities,statementData.balanceSheet.sharesBasic,statementData.balanceSheet.deposits,statementData.balanceSheet.acctRec,statementData.balanceSheet.intangibles,statementData.balanceSheet.liabilitiesNonCurrent,statementData.balanceSheet.totalLiabilities,statementData.balanceSheet.liabilitiesCurrent,statementData.cashFlow.ncfx,statementData.cashFlow.ncf,statementData.cashFlow.issrepayEquity,statementData.cashFlow.depamor,statementData.cashFlow.capex,statementData.cashFlow.businessAcqDisposals,statementData.cashFlow.ncff,statementData.cashFlow.sbcomp,statementData.cashFlow.issrepayDebt,statementData.cashFlow.ncfi,statementData.cashFlow.investmentsAcqDisposals,statementData.cashFlow.ncfo,statementData.cashFlow.freeCashFlow,statementData.cashFlow.payDiv,statementData.incomeStatement.shareswaDil,statementData.incomeStatement.opex,statementData.incomeStatement.ebt,statementData.incomeStatement.rnd,statementData.incomeStatement.ebit,statementData.incomeStatement.eps,statementData.incomeStatement.netinc,statementData.incomeStatement.intexp,statementData.incomeStatement.nonControllingInterests,statementData.incomeStatement.revenue,statementData.incomeStatement.opinc,statementData.incomeStatement.netIncDiscOps,statementData.incomeStatement.grossProfit,statementData.incomeStatement.prefDVDs,statementData.incomeStatement.taxExp,statementData.incomeStatement.epsDil,statementData.incomeStatement.ebitda,statementData.incomeStatement.shareswa,statementData.incomeStatement.netIncComStock,statementData.incomeStatement.sga,statementData.incomeStatement.consolidatedIncome,statementData.incomeStatement.costRev,statementData.overview.currentRatio,statementData.overview.revenueQoQ,statementData.overview.bookVal,statementData.overview.roe,statementData.overview.rps,statementData.overview.longTermDebtEquity,statementData.overview.profitMargin,statementData.overview.piotroskiFScore,statementData.overview.grossMargin,statementData.overview.roa,statementData.overview.epsQoQ,statementData.overview.bvps
0,2020-12-26 00:00:00+00:00,1,2021,1.790000e+08,1.187450e+11,0.0,1.430100e+10,4.081600e+10,1.276200e+10,3.540540e+11,1.999480e+11,1.541060e+11,3.601000e+10,7.395000e+09,6.622400e+10,3.793300e+10,9.928100e+10,4.973000e+09,6.384600e+10,1.595610e+11,1.120430e+11,0.0,1.700180e+10,0.0,5.862000e+10,0.0,1.553230e+11,2.878300e+11,1.325070e+11,0.0,-2.070000e+09,-2.477500e+10,2.666000e+09,-3.500000e+09,-9000000.0,-3.224900e+10,2.020000e+09,-9.780000e+08,-8.584000e+09,-5.279000e+09,3.876300e+10,3.526300e+10,-3.613000e+09,1.711369e+10,1.079400e+10,3.357900e+10,5.163000e+09,3.357900e+10,1.700,2.875500e+10,0.0,0.0,1.114390e+11,3.353400e+10,0.0,4.432800e+10,0.0,4.824000e+09,1.680,3.624500e+10,1.693512e+10,2.875500e+10,5.631000e+09,2.875500e+10,6.711100e+10,1.163003,0.213681,6.622400e+10,0.905941,6.554541,1.499169,0.397778,7.0,0.397778,0.194362,0.349206,3.895117
1,2020-09-26 00:00:00+00:00,4,2020,-4.060000e+08,1.008870e+11,0.0,1.496600e+10,5.292700e+10,1.376900e+10,3.238880e+11,1.801750e+11,1.437130e+11,3.801600e+10,6.643000e+09,6.533900e+10,3.676600e+10,9.866700e+10,4.061000e+09,4.229600e+10,1.538140e+11,1.124360e+11,0.0,1.710254e+10,0.0,3.744500e+10,0.0,1.531570e+11,2.585490e+11,1.053920e+11,0.0,4.750000e+09,-1.673700e+10,2.702000e+09,-

Let's clean up and make two functions out of above snippets

In [39]:
def download_csv_data(symbol, start_date, end_date=None, freq='daily', path=None):
    df_path=os.path.join(path, f'{symbol}.csv')
    fund_df_path=os.path.join(path, f'{symbol}_fund.csv')
    
    if os.path.exists(df_path):
#         print(f'{symbol}.csv exists')
        return
    
    df = pd.DataFrame(get_price(symbol, start_date, end_date, freq))
    df2 = pd.DataFrame(get_daily_fundamental(symbol, start_date, end_date))
    df3 = normalize_fundamental_quarterly_data(get_quarterly_fundamental(symbol, start_date, end_date))
    df3 = df3[~df3['quarter'].isin([0])]    # Remove the records that quarter = 0
    
    df['date'] = pd.to_datetime(df['date'], utc=True)
    df2['date'] = pd.to_datetime(df2['date'], utc=True)
    df3['date'] = pd.to_datetime(df3['date'], utc=True)

    dd = pd.merge(df, df2, on='date', how='left')
    dd = dd.ffill()
    df3 = df3.ffill()
    df3 = df3[~df3['quarter'].isin([0])]

    dd.to_csv(df_path, header=True, index=True)
    df3.to_csv(fund_df_path, header=True, index=True)
        
    del df
    del df2
    del df3
    del dd

def load_data(symbol, start_date, end_date=None, freq='daily', path=None):
    df_path=os.path.join(path, f'{symbol}.csv')
    fund_df_path=os.path.join(path, f'{symbol}_fund.csv')
    
    try:
        df = pd.read_csv(df_path, index_col=0)
        fund_df = pd.read_csv(fund_df_path, index_col=0)
    except Exception as e:
        raise(e)
        
    return df, fund_df

## We're almost set

Above are pretty much the self-defined functions for retrieving the data we need. Let's close this section by downloading all the data we need.

### Get the stock symbols

In [38]:
# Get stock list from S&P 500
# Needs VPN
stock_list = get_snp_stock_list()

### Start downloading the data we need

In [42]:
# Get meta data dataframe
meta_df = pd.json_normalize(load_meta())
meta_df['sicCode'].fillna(0, inplace=True)

# Download data
for symbol in notebook.tqdm(stock_list):
    print(f'Start processing {symbol}...')
    download_csv_data(
        symbol=symbol,
        start_date='2018-07-31',
        end_date='2021-01-31',
        freq='daily',
        path='[Where you store your csv files]'
    )

Start processing mmm...
Start processing abt...
Start processing abbv...
Start processing abmd...
Start processing acn...
Start processing atvi...
Start processing adbe...
Start processing amd...
Start processing aap...
Start processing aes...
Start processing afl...
Start processing a...
Start processing apd...
Start processing akam...
Start processing alk...
Start processing alb...
Start processing are...
Start processing alxn...
Start processing algn...
Start processing alle...
Start processing lnt...
Start processing all...
Start processing googl...
Start processing mo...
Start processing amzn...
Start processing amcr...
Start processing aee...
Start processing aal...
Start processing aep...
Start processing axp...
Start processing aig...
Start processing amt...
Start processing awk...
Start processing amp...
Start processing abc...
Start processing ame...
Start processing amgn...
Start processing aph...
Start processing adi...
Start processing anss...
Start processing antm...
Star

Nice! We have all the data downloaded.